In [1]:
import io
from IPython.display import clear_output
import regex
from opencc import OpenCC
from gensim.models import KeyedVectors
import math
from os import listdir
from os.path import isfile, isdir, join
import os
import requests

In [2]:
#festtest_chinese讀檔
def load_vectors(fname):
    global AL_CBOW_Data
    print('初始化中')
    AL_CBOW_Data = KeyedVectors.load_word2vec_format('cc.zh.300.vec')
    clear_output(wait=True)
    print('完成')
    return


In [22]:
#cosine距離運算
def cosine_meansure(word1,word2):
    word1 = OpenCC('tw2s').convert(word1)
    word2 = OpenCC('tw2s').convert(word2)
    #判斷此兩個詞是否有出現過
    if (not word1 in AL_CBOW_Data) :
        print('We do not have ['+word1+']')
    if (not word2 in AL_CBOW_Data) :
        print('We do not have ['+word2+']')
    if(not word1 in AL_CBOW_Data) or (not word2 in AL_CBOW_Data) :
        #print('出現此訊息，代表因為我們產生Word2Vec的資料庫沒有這個詞彙，所以無法計算!')
        return 0.0
    #開始計算餘弦相關性
    molecule_A = 0.0 #詞彙1的分母計算
    molecule_B = 0.0 #詞彙2的分母計算
    Denominator = 0.0 #交互的分子計算
    for index in range(len(AL_CBOW_Data[word1])):
        Denominator += AL_CBOW_Data[word1][index] * AL_CBOW_Data[word2][index]
        molecule_A += AL_CBOW_Data[word1][index] * AL_CBOW_Data[word1][index]
        molecule_B += AL_CBOW_Data[word2][index] * AL_CBOW_Data[word2][index]
    molecule_all = math.sqrt(molecule_A)*math.sqrt(molecule_B)
    return (Denominator / molecule_all)


In [24]:
#先把題組間的距離算好
def ready_for_distance(dictionary_a2p,key):#dictionary_a2p is list;key = the answer
    data_a2p = []
    data_p2p = [[0 for i in range(len(dictionary_a2p))] for j in range(len(dictionary_a2p))]
    
    count = 0
    print('計算' + key)
    for value in dictionary_a2p:
        data_a2p.append (cosine_meansure(key,value))
        #print('第'+str(count)+'數'+value+'與a：'+str(data_a2p[count]))
        count += 1
    
    for i in range(len(dictionary_a2p)):
        clear_output(wait=True)
        print('計算' +'第'+str(i)+'數'+ dictionary_a2p[i])
        for j in range(i+1,len(dictionary_a2p)):
            data_p2p[i][j] = cosine_meansure(dictionary_a2p[i],dictionary_a2p[j])
            #print('第'+str(i)+'數'+dictionary_a2p[i]+'與'+str(j)+'數'+dictionary_a2p[j]+'：'+str(data_p2p[i][j]))
    
    return data_a2p , data_p2p

In [21]:
#好題目評分
def standard( atp1, atp2, atp3, p1tp2, p1tp3, p2tp3):
    sum_a2p = atp1 + atp2 + atp3
    sum_p2p = p1tp2 + p1tp3 + p2tp3
    standard_ = sum_a2p - sum_p2p
    return standard_

In [62]:
#整理題目分數大小，並輸出
def datasort(ans_list,data,name,data_a2p,data_p2p): #data須為dict類別,依照values排列順序
    print(data)
    data.sort(key=lambda x: x[3], reverse=True)
    wp = open("./standard/sum/ptp權重x1/"+name+".txt","w",encoding="utf-8")
    wp.write('\t\t\t\tstand\tatp1\tatp2\tatp3\tp1tp2\tp1tp3\tp2tp3\n')
    for num in range(0,len(data)):
        wp.write(ans_list[data[num][0]]+"\t"+ans_list[data[num][1]]+"\t"+ans_list[data[num][2]]+"\t"+name+"\t"+str(round(data[num][3],3))+"\t"+str(round(data_a2p[data[num][0]],3))+"\t"+str(round(data_a2p[data[num][1]],3))+"\t"+str(round(data_a2p[data[num][2]],3))+"\t"+str(round(data_p2p[data[num][0]][data[num][1]],3))+"\t"+str(round(data_p2p[data[num][0]][data[num][2]],3))+"\t"+str(round(data_p2p[data[num][1]][data[num][2]],3))+"\n")
                 #str(round(data[num][3],3))+"\t"+str(round(data_a2p[data[num][0]],3))+" "+str(round(data_a2p[data[num][1]],3))+" "+str(round(data_a2p[data[num][2]],3))+" "+str(round(data_p2p[data[num][0]][data[num][1]],3))+" "+str(round(data_p2p[data[num][0]][data[num][2]],3))+" "+str(round(data_p2p[data[num][1]][data[num][2]],3))+"\n")
    wp.close

In [37]:
def thresh_hole_check(distance , thresh):
    if(thresh[0]>distance or thresh[1]<distance):
        return True
    else:
        return False 

In [38]:
#找該題目的組合
def find_a2p_pair(dictionary_a2p , data_a2p , data_p2p , key , thresh_hole):
    
    a2p_pairdata = []
    count_line = 0
    problem_num = 0
    print(key+' 處理中')
    if(not key in AL_CBOW_Data):
        return
    for p1 in range(len(dictionary_a2p)-2):
        if(not dictionary_a2p[p1] in AL_CBOW_Data):
            print(dictionary_a2p[p1]+'out')
            continue
        #
        atp1 = data_a2p[p1]
        if(thresh_hole_check(atp1 ,thresh_hole[0])):
            print(dictionary_a2p[p1]+'bang')
            continue
        
        for p2 in range(p1+1,len(dictionary_a2p)-1):
            if(not dictionary_a2p[p2] in AL_CBOW_Data):
                print(dictionary_a2p[p2]+'out')
                continue
            #
            atp2 = data_a2p[p2]
            p1tp2 = data_p2p[p1][p2]
            #
            if(atp1 < p1tp2 or atp2 < p1tp2 or thresh_hole_check(atp2 ,thresh_hole[0]) or thresh_hole_check(p1tp2 ,thresh_hole[1])):
                count_line+=1
                continue

            for p3 in range(p2+1,len(dictionary_a2p)):
                if(not dictionary_a2p[p3] in AL_CBOW_Data):
                    print(dictionary_a2p[p3]+'out')
                    continue
                #
                atp3 = data_a2p[p3]
                p1tp3 = data_p2p[p1][p3]
                p2tp3 = data_p2p[p2][p3]
                #
                if(atp1 < p1tp3 or atp3 < p1tp3 or atp2 < p2tp3 or atp3 < p2tp3 or thresh_hole_check(atp3 ,thresh_hole[0]) or thresh_hole_check(p1tp3 ,thresh_hole[1]) or thresh_hole_check(p2tp3 ,thresh_hole[1])):
                    count_line+=1
                    continue

                standard_all = standard( atp1, atp2, atp3, p1tp2, p1tp3, p2tp3)
                a2p_pairdata.append ((p1 , p2 , p3 , standard_all))

                #顯示進度
                problem_num += 1
                count_line+=1
                if(count_line%1000 == 0):
                    #clear_output(wait=True)
                    print(key+' 處理中 '+str(count_line))
                    print('共有'+str(problem_num)+'題\n')
                    print(str(p1)+" "+str(p2)+" "+str(p3))
                #顯示完畢

    #clear_output(wait=True)
    print(key +' 整理！'+str(problem_num)+'/'+str(count_line))
    datasort(dictionary_a2p , a2p_pairdata , key, data_a2p, data_p2p)
    print('完成：'+key)
                    
    return
                    

In [39]:
#主程式之，找所有題目的題組
def frequency_database_a2p_all():
    pattern = '([\p{Han}]+):' #抓漢字的正則運算法
    
    fp = open("../詞頻聯想/修正2頻3心像表格.txt", "r" ,encoding="utf-8",errors="ignore")
    
    print('載入...')
    line = fp.readline()
    line = fp.readline()
    while(line):
        dictionary_a2p = []
        key = regex.findall(pattern, line) #抓詞頻答案
        line = fp.readline()
        line = fp.readline()
        line = fp.readline()
        ans = regex.findall(pattern, line) #抓詞頻題組
        dictionary_a2p = ans[2:]

        clear_output(wait=True)
        print('算距離階段')
        data_a2p , data_p2p = ready_for_distance(dictionary_a2p,key[0])

        clear_output(wait=True)
        print('找題目階段')
        find_a2p_pair(dictionary_a2p , data_a2p , data_p2p , key[0])

        line = fp.readline()
        clear_output(wait=True)
        print('完成')
    
    fp.close
    return

In [43]:
#先將題庫對應題組做成dictionary
def initial_frequency_database():
    pattern_1 = '([\p{Han}]+)'
    pattern_2 = '([\p{Han}]+):' #抓漢字的正則運算法
    global frequency_database
    frequency_database = {}
    fp = open("../詞頻聯想/修正2頻3心像表格.txt", "r" ,encoding="utf-8",errors="ignore")
    
    print('載入...')
    line = fp.readline()
    line = fp.readline()
    
    while(line):
        key = regex.findall(pattern_1, line) #抓詞頻答案
        line = fp.readline()
        line = fp.readline()
        line = fp.readline()
        ans = regex.findall(pattern_2, line) #抓詞頻題組
        frequency_database[key[0]] = ans[2:]
        
        line = fp.readline()
    
    clear_output(wait=True)
    print('完成')
    fp.close

In [41]:
#主程式之二：指定特定詞來做題組
def assign_frequency_database(key, thresh_hole):
    if not key in AL_CBOW_Data:
        print('無此字')
        return
    dictionary_a2p = frequency_database[key]
    clear_output(wait=True)
    print('算距離階段')
    data_a2p , data_p2p = ready_for_distance(dictionary_a2p,key)

    clear_output(wait=True)
    print('找題目階段')
    find_a2p_pair(dictionary_a2p , data_a2p , data_p2p , key , thresh_hole)

    #clear_output(wait=True)
    print('完成')
    
    return

In [65]:
def run_all(thresh_hole):
    for key in frequency_database:
        assign_frequency_database(key, thresh_hole)

In [6]:
#load 向量集
data = load_vectors('cc.zh.300.vec')

完成


In [7]:
#主程式之，找所有題目的題組
#frequency_database_a2p()

NameError: name 'frequency_database_a2p' is not defined

In [44]:
#初始化題庫
initial_frequency_database()

完成


In [69]:
thresh_hole = [[0.25,0.5],[0,0.25]]

In [63]:
#主程式之二：指定特定詞來做題組
assign_frequency_database('月亮' , thresh_hole)

找題目階段
月亮 處理中
星星bang
圓缺bang
圓滿bang
月亮 整理！148/527
[(0, 2, 6, 0.7158824144648623), (0, 2, 9, 0.682968286781252), (0, 2, 12, 0.7458644115487467), (0, 2, 14, 0.6975209776898945), (0, 6, 9, 0.6786242750794294), (0, 6, 10, 0.6129826811871047), (0, 6, 16, 0.7513074395885968), (0, 6, 18, 0.6464460318063923), (0, 7, 9, 0.6367131019718386), (0, 7, 10, 0.7159406822931244), (0, 7, 12, 0.6511599000891382), (0, 7, 14, 0.7813160171687192), (0, 9, 10, 0.5733977042111789), (0, 9, 12, 0.5176750325623457), (0, 9, 14, 0.7493289015704376), (0, 9, 16, 0.6640562758937373), (0, 10, 12, 0.6107041367895656), (0, 10, 18, 0.5112911079106335), (0, 10, 19, 0.7038109681882336), (0, 12, 14, 0.7542692603009558), (0, 12, 16, 0.7029215002621105), (0, 12, 18, 0.5951986213989111), (0, 12, 19, 0.6851237494762317), (0, 14, 16, 0.7368395868889577), (0, 14, 18, 0.663975802948692), (0, 14, 19, 0.8074938212139274), (0, 16, 19, 0.6858289569857676), (2, 6, 8, 0.6759516668771464), (2, 6, 9, 0.7483689128076197), (2, 6, 15, 0.5108426

In [33]:
cosine_meansure('月亮','耳朵')

0.3492052215166766

In [70]:
run_all(thresh_hole)

找題目階段
眼睛 處理中
鼻子bang
耳朵bang
嘴巴bang
大bang
水汪汪bang
視力bang
瞳孔bang
眼睛 整理！244/1058
[(0, 1, 12, 0.5450485880604925), (0, 1, 24, 0.720515808609884), (0, 1, 28, 0.6561364485272276), (0, 12, 19, 0.38342671188610666), (0, 12, 28, 0.5382683894127005), (0, 18, 19, 0.6252606721761595), (0, 18, 22, 0.5944037115335834), (0, 18, 28, 0.7297664077343415), (0, 19, 20, 0.3774653100582007), (0, 19, 22, 0.42347067561320983), (0, 19, 24, 0.5717912741380565), (0, 19, 28, 0.6084235028013063), (0, 20, 22, 0.4096291127511553), (0, 20, 24, 0.4897723093681975), (0, 20, 28, 0.528001031140337), (0, 22, 24, 0.5137275365404413), (0, 22, 28, 0.598726549074744), (0, 23, 24, 0.6313534301759297), (0, 23, 28, 0.6363779886258523), (0, 24, 28, 0.6905052980471176), (1, 3, 5, 0.644517838226605), (1, 3, 8, 0.5408958732803576), (1, 3, 27, 0.5140798206805508), (1, 3, 28, 0.4654074100694858), (1, 5, 12, 0.5789094192903003), (1, 5, 24, 0.7329532734787214), (1, 5, 28, 0.6590396838059556), (1, 8, 11, 0.7980075313083136), (1, 8, 12, 0.

完成


In [17]:
print(1<2)

True
